# Descripción del proyecto  

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.
Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del curso de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.
Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

## Instrucciones del proyecto.
1. Abre y examina el archivo de datos. Dirección al archivo:datasets/users_behavior.csv 
2. Segmenta los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.
3. Investiga la calidad de diferentes modelos cambiando los hiperparámetros. Describe brevemente los hallazgos del estudio.
4. Comprueba la calidad del modelo usando el conjunto de prueba.
5. Tarea adicional: haz una prueba de cordura al modelo. Estos datos son más complejos que los que habías usado antes así que no será una tarea fácil. Más adelante lo veremos con más detalle.

## Descripción de datos
Cada observación en el dataset contiene información del comportamiento mensual sobre un usuario. La información dada es la siguiente:
- сalls — número de llamadas,
- minutes — duración total de la llamada en minutos,
- messages — número de mensajes de texto,
- mb_used — Tráfico de Internet utilizado en MB,
- is_ultra — plan para el mes actual (Ultra - 1, Smart - 0).

## Cargar librerías

In [65]:
#Cargar todas las librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

## Cargar datos

In [80]:
df=pd.read_csv('datasets/users_behavior.csv')
df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/users_behavior.csv'

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [68]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Haciendo una visualización inicial de los datos, tenemos 5 columnas y 3214 filas, de los cuales a simple vista no vemos ningún ausente. Ya veremos más a fondo esto. 

In [69]:
#Calculamos los ausentes
print('Ausentes:\n',df.isna().sum())

Ausentes:
 calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64


In [70]:
#Calculamos los duplicados
print('Duplicados:\n',df.duplicated().sum())


Duplicados:
 0


Podemos evidenciar que no contamos con valores ausentes ni valores duplicados en el dataset y tenemos datos limpios listos para el modelo de clasificación.

## Segmentación de los datos

In [71]:
#Usamos la semilla para los datos pseudoaleatorios
seed=54321
#Segmentamos primero los datos de entrenamiento y los datos de prueba
df_train,df_test=train_test_split(df,test_size=0.2,random_state=seed)
features=df_train.drop('is_ultra',axis=1)
target=df_train['is_ultra']

In [72]:
#Segmentamos ahora los datos de entrenamiento y validación
features_train,features_valid,target_train,target_valid=train_test_split(features,target,test_size=0.2,random_state=seed)

Ahora tenemos 80% de datos de entrenamiento, 20% de datos de validación y 20% de datos de prueba. A continuación vamos a entrenar los modelos.

## Prueba de modelos de clasificación

### Arbol de desición

In [73]:
best_score_1=0
best_depth_1=0
for depth in range(1,11):
    model_1=DecisionTreeClassifier(random_state=seed,max_depth=depth)
    model_1.fit(features_train,target_train)
    prediction_1=model_1.predict(features_valid)
    result_1=metrics.accuracy_score(target_valid,prediction_1)
    if result_1 > best_score_1:
        best_score_1=result_1
        best_depth_1=depth
print(f'max_depth = {depth} : {result_1}')


max_depth = 10 : 0.7805825242718447


Obtenemos una exactitud del 78% en el arbol de desición con una profundidad de 10 ramas en el arbol, sin embargo, es una profundidad muy alta que puede costarnos un sobreentrenamiento.

### Bosque Aleatorio

In [74]:
best_depth_2=0
best_estimator_2=0
best_score_2=0
for est in range(1,51,10):
    for depth in range(1,10):
        model_2=RandomForestClassifier(random_state=seed,max_depth=depth,n_estimators=est)
        model_2.fit(features_train,target_train)
        prediction_2 = model_2.predict(features_valid)
        result_2=metrics.accuracy_score(target_valid,prediction_2)
        if result_2 > best_score_2:
            best_score_2=result_2
            best_estimator_2=est
            best_depth_2=depth
print(f'El mejor resultado es {best_score_2} con {best_estimator_2} estimadores y {best_depth_2} de profundidad.')


El mejor resultado es 0.8233009708737864 con 11 estimadores y 4 de profundidad.


El mejor resultado hasta el momento es el bosque aleatorio con 11 estimadores y 4 de profundidad del arbol. Al tener varios estimadores, no nos preocupa el sobreentrenamiento, además la exactitud es de 82%.

### Regresión Logística

In [75]:
model_3=LogisticRegression(random_state=seed,solver='liblinear')
model_3.fit(features_train,target_train)
result_3_train=model_3.score(features_train,target_train)
result_3_valid=model_3.score(features_valid,target_valid)
print(f'Resultado entrenamiento: {result_3_train}\nResultado validación: {result_3_valid}')

Resultado entrenamiento: 0.7217898832684825
Resultado validación: 0.7184466019417476


La regresión logística es un algoritmo rápido, sin embargo, tiene el porcentaje de exactitud más bajo con 71,8%.

El modelo escogido para la prueba con el conjunto de prueba es el **bosque aleatorio con 4 de profundidad y 11 estimadores.**

### Evaluación del modelo

In [76]:
features_test=df_test.drop('is_ultra',axis=1)
target_test=df_test['is_ultra']

In [77]:
model_final=RandomForestClassifier(random_state=seed,max_depth=4,n_estimators=11)
model_final.fit(features,target)
prediction_test=model_final.predict(features_test)
result_final=metrics.accuracy_score(target_test,prediction_test)
print(f'El resultado final tiene una exactitud de {result_final*100}%.')

El resultado final tiene una exactitud de 77.76049766718506%.


Al evaluar el modelo obtenemos una exactitud del 77,7%, mayor a lo esperado que era 75%, por lo cual podemos concluír que el algoritmo es adecuado para la predicción del dataset de prueba.

### Prueba de cordura

In [78]:
print(metrics.classification_report(target_test, prediction_test))

              precision    recall  f1-score   support

           0       0.77      0.94      0.85       427
           1       0.79      0.46      0.58       216

    accuracy                           0.78       643
   macro avg       0.78      0.70      0.71       643
weighted avg       0.78      0.78      0.76       643



In [79]:
#Porcentaje de 0 correctos: 0.77
#Porcentaje de 1 correctos: 0.79
#exactitud= 0.5*(Porcentaje de 0 correctos)+0.5*(Porcentaje de 1 correctos)
print('La calidad del modelo es de: ',(0.5*0.77)+(0.5*0.79))

La calidad del modelo es de:  0.78


Al realizar la prueba de cordura, podemos ver que la calidad del modelo es alta, **0.78**.

## Conclusión

El mejor modelo de clasificación es el **Bosque Aleatorio** con un **82%** de exactitud con el dataset de validación y **77%** con el modelo de prueba.

Si análizamos la exactitud del modelo, podemos ver que el modelo adivinó el 79% de los usuarios con plan Ultra y el 77% de los usuarios con plan Smart.

Además, tenemos un recall de 94% de los de plan ultra y 46% de plan smart, lo que significa que predijo más cantidad de personas con el plan ultra y menos con el plan smart, por lo que tenemos una oportunidad de mejora frente a esto.

Finalmente al hacer la prueba de cordura, tenemos un 0.78 de calidad del modelo, lo que nos indica que tenemos un buen modelo.